# Demo: Serenity Portfolio Valuation API

In the coming release of Serenity we will support various portfolio analytics, but in the current release we support just basic NAV calculations given a long-only or long/short portfolio composed of assets from the Serenity Asset Master.

In [ ]:
%%capture --no-stderr --no-display
%load_ext autoreload
%autoreload 2

In [ ]:
import os
from serenity_sdk.widgets import ConnectWidget

# if you want to auto-connect, set this environment variable to your desired default
connect_widget = ConnectWidget(os.getenv('SERENITY_CONFIG_ID', None))

In [ ]:
import datetime

import pandas as pd

# create an alias to the api
api = connect_widget.get_api()

To value a portfolio the first thing we need is a portfolio:

In [ ]:
asset_master = api.refdata().load_asset_master()

portfolio_raw = {
    'ADA': 1000000,
    'BTC': 100,
    'ETH': 1000,
    'XRP': 2000000,
    'ALGO': 1500000,
    'SOL': 10000,
    'DOT': 50000
}
portfolio = asset_master.create_portfolio(portfolio_raw, symbology='NATIVE')

Similar to Risk and VaR with `CalculationContext`, we create a `PricingContext` to set up the valuator. Here we override one of the parameters to specify that we want to treat both USD and USD-pegged stablecoins as cash positions:

In [ ]:
from datetime import date
from serenity_sdk.types.common import CashTreatment, PricingContext

usd_asset_id = asset_master.get_asset_id_by_symbol('ccy.usd', 'SERENITY')
ctx = PricingContext(as_of_date=date(2021, 7, 1), cash_treatment=CashTreatment.FIAT_PEGGED_STABLECOINS, base_currency_id=usd_asset_id)

Now we can value the portfolio. As this one only has non-stablecoin tokens, we expect a zero cash position:

In [ ]:
result = api.valuation().compute_portfolio_value(ctx, portfolio)
result.close.cash_position_value

What if we create a portfolio with a mix of stablecoins and fiat? And because Solana is always crashing, why don't we short it.

In [ ]:
portfolio_raw = {
    'ADA': 1000000,
    'BTC': 100,
    'ETH': 1000,
    'XRP': 2000000,
    'ALGO': 1500000,
    'SOL': -10000,
    'DOT': 50000,
    'USDC': 100000,
    'USD': 250000
}
portfolio = asset_master.create_portfolio(portfolio_raw, symbology='NATIVE')

In [ ]:
result = api.valuation().compute_portfolio_value(ctx, portfolio)
result.close.cash_position_value

OK, but we learned the hard way in the spring of 2022 that USD-pegged stablecoins and USD are not always the same. How about we value the portfolio with the FIAT_ONLY model instead to be safe?

In [ ]:
ctx.cash_treatment = CashTreatment.FIAT_ONLY
result = api.valuation().compute_portfolio_value(ctx, portfolio)
result.close.cash_position_value

That looks better. Let's summarize the portfolio's value T and T-1 in a table; note that in Serenity we mark to a specific mark time like NY_EOD, and so the close and previous times will be the most recent NY_EOD close and the previous NY_EOD close:

In [ ]:
rows = []
rows.append({
   'closeTime': 'T (close)',
   'netHoldings': result.close.net_holdings_value,
   'grossHoldings': result.close.gross_holdings_value,
   'cashPosition': result.close.cash_position_value,
   'netAssetValue': result.close.net_asset_value
})
rows.append({
   'closeTime': 'T-1 (previous)',
   'netHoldings': result.previous.net_holdings_value,
   'grossHoldings': result.previous.gross_holdings_value,
   'cashPosition': result.previous.cash_position_value,
   'netAssetValue': result.previous.net_asset_value
})
df = pd.DataFrame(rows)
df.set_index('closeTime', inplace=True)
df.style.format(lambda val: f'${val:,.2f}')